In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('final_crawled_api_data.csv')
df.head()

,title,link,mountain,upload_to,운해 여부,위치,전날 기온,전날 강수량,전날 습도,전날 기압,일출 기온,일출 풍속,일출 습도,일출 기압,일출 전운량,일교차
0,오늘 일출과 운해,https://cafe.naver.com/windstopper/ArticleRead...,[‘심항산’],20230508.0,1.0,충주,5.8,0.0,84.0,1001.4,3.2,0.0,93.0,1003.0,0.0,-2.6
1,"노동절 새벽, 월출산의 운해일출",https://cafe.naver.com/windstopper/ArticleRead...,['월출산'],20230501.0,1.0,강진,8.9,0.0,91.0,1012.2,9.3,0.8,92.0,1012.4,7.0,0.4
2,백운대 일출산행(feat. 운해사냥),https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230501.0,1.0,서울,12.1,0.0,87.0,1001.3,10.1,2.4,91.0,1002.7,7.0,-2.0
3,백운대 운해사냥성공(thanks to또민아빠님),https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230430.0,1.0,서울,10.0,0.0,83.0,998.3,7.9,2.5,87.0,998.9,0.0,-2.1
4,지금 이 순간(역대급 제비봉 운해),https://cafe.naver.com/windstopper/ArticleRead...,[‘제비봉’],20230427.0,1.0,제천,1.7,0.0,100.0,987.3,-0.3,0.0,100.0,989.1,10.0,-2.0


In [3]:
df=df.drop(['title', 'link', 'upload_to', 'mountain', '위치'],axis=1)
df

,운해 여부,전날 기온,전날 강수량,전날 습도,전날 기압,일출 기온,일출 풍속,일출 습도,일출 기압,일출 전운량,일교차
0,1.0,5.8,0.0,84.0,1001.4,3.2,0.0,93.0,1003.0,0.0,-2.6
1,1.0,8.9,0.0,91.0,1012.2,9.3,0.8,92.0,1012.4,7.0,0.4
2,1.0,12.1,0.0,87.0,1001.3,10.1,2.4,91.0,1002.7,7.0,-2.0
3,1.0,10.0,0.0,83.0,998.3,7.9,2.5,87.0,998.9,0.0,-2.1
4,1.0,1.7,0.0,100.0,987.3,-0.3,0.0,100.0,989.1,10.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...
802,0.0,-10.8,0.0,89.0,941.9,-13.1,0.3,87.0,941.8,0.0,-2.3
803,0.0,-4.5,0.0,61.0,1008.7,-7.0,0.6,72.0,1008.7,0.0,-2.5
804,0.0,8.5,0.0,76.0,1015.7,7.7,0.3,85.0,1015.7,0.0,-0.8
805,0.0,-5.0,0.0,73.0,1009.7,-5.8,0.0,83.0,1009.7,0.0,-0.8


In [4]:
df=df.dropna()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 805
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   운해 여부   804 non-null    float64
 1   전날 기온   804 non-null    float64
 2   전날 강수량  804 non-null    float64
 3   전날 습도   804 non-null    float64
 4   전날 기압   804 non-null    float64
 5   일출 기온   804 non-null    float64
 6   일출 풍속   804 non-null    float64
 7   일출 습도   804 non-null    float64
 8   일출 기압   804 non-null    float64
 9   일출 전운량  804 non-null    float64
 10  일교차     804 non-null    float64
dtypes: float64(11)
memory usage: 75.4 KB


In [7]:
from sklearn.model_selection import train_test_split

feature_columns =list(df.columns.difference(['운해 여부']))

X = df[feature_columns]
y = df['운해 여부']

train_x, test_x, train_y, test_y = train_test_split(X,y,stratify=y, test_size=0.2, random_state=42)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(643, 10) (161, 10) (643,) (161,)


In [8]:
test_y.info()

<class 'pandas.core.series.Series'>
Int64Index: 161 entries, 296 to 726
Series name: 운해 여부
Non-Null Count  Dtype  
--------------  -----  
161 non-null    float64
dtypes: float64(1)
memory usage: 2.5 KB


## Adaboost
이전 학습기가 잘못 예측한 샘플에 가중치를 부여하여 다음 학습기를 학습합니다. 오분류된 샘플에 더 집중하여 학습기를 조정

오분류된 샘플에 대해 가중치를 부여하고, 다음 학습기의 오차를 최소화하는 방향으로 학습

In [21]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_model =DecisionTreeClassifier(max_depth=1)
clf =AdaBoostClassifier(base_estimator = tree_model, n_estimators =1000, random_state=0)
clf.fit(train_x, train_y)
pred=clf.predict(test_x)
print(accuracy_score(test_y, pred))

C:\Users\leeji\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.6956521739130435


## GradientBoost

이전 학습기의 잔차(Residual)에 대해 새로운 학습기를 학습시킵니다. 잔차를 예측하는 새로운 학습기를 추가함으로써 예측 성능을 개선하는 방식

예측값과 실제값 사이의 잔차(Residual)에 대해 새로운 학습기를 학습시킵니다. 새로운 학습기는 잔차를 예측하는 방향으로 학습

In [23]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from math import sqrt

clf2 = GradientBoostingClassifier(n_estimators=100, random_state=0)
clf2.fit(train_x, train_y)
pred=clf2.predict(test_x)
print(accuracy_score(test_y, pred))

0.6956521739130435


## 예측값 확률 구하기 

In [11]:
print(pred)

[0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1.
 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1.]


In [12]:
id=test_x.index
type(id)
id=pd.Series(id)
type(id)

pandas.core.series.Series

In [13]:
pred_proba = clf2.predict_proba(test_x)[:, 1]


In [14]:
type(pred_proba)

numpy.ndarray

In [15]:
#pred_proba=pd.Series(pred_proba)

testy=test_y.values
print(testy)
testy=pd.Series(test_y)
print(type(testy))
print(testy.shape)

testy=testy.set_index()

In [16]:
print(pred_proba)
pred_proba=pd.Series(pred_proba)
print(type(pred_proba))
print(pred_proba.shape)

[0.20072925 0.47859302 0.81289849 0.33699391 0.81484502 0.56186741
 0.03425377 0.84967841 0.31615855 0.13145943 0.03762176 0.23993562
 0.13834115 0.17130027 0.4859871  0.1217817  0.16131055 0.16300897
 0.11791936 0.09868248 0.11329102 0.03148094 0.1566909  0.55312174
 0.07469835 0.96672214 0.65581999 0.297486   0.64763221 0.22878391
 0.04062652 0.3991989  0.69137807 0.76645922 0.34087371 0.62950829
 0.41904788 0.0931898  0.61506438 0.11851825 0.55914406 0.24517662
 0.11851825 0.86131856 0.14843075 0.81417337 0.07235754 0.03976495
 0.54211515 0.14553262 0.48168851 0.33537406 0.77797366 0.22986471
 0.43890575 0.74442279 0.76359946 0.67760206 0.13224727 0.77389575
 0.64657633 0.24517662 0.35321475 0.15925715 0.87612355 0.06891695
 0.04121358 0.2769243  0.13834115 0.72288907 0.24295047 0.93601797
 0.12236315 0.52780771 0.28217654 0.73245413 0.66428423 0.23031768
 0.20518981 0.06259204 0.26572087 0.29397313 0.03024195 0.73235701
 0.24295047 0.54211515 0.73129693 0.0669018  0.70429865 0.1671

result = pd.concat([pred_proba, testy])
result

## 데이터 정규화 min-max

In [18]:
from sklearn.preprocessing import MinMaxScaler

# min-max 정규화 객체 생성
scaler = MinMaxScaler()

# min-max 정규화 적용
for i in feature_columns:
    df[i] = scaler.fit_transform(df[[i]])

print(df)

     운해 여부     전날 기온  전날 강수량     전날 습도     전날 기압     일출 기온     일출 풍속  \
0      1.0  0.497717     0.0  0.811765  0.698236  0.468750  0.000000   
1      1.0  0.568493     0.0  0.894118  0.798514  0.604911  0.119403   
2      1.0  0.641553     0.0  0.847059  0.697307  0.622768  0.358209   
3      1.0  0.593607     0.0  0.800000  0.669452  0.573661  0.373134   
4      1.0  0.404110     0.0  1.000000  0.567317  0.390625  0.000000   
..     ...       ...     ...       ...       ...       ...       ...   
801    1.0  0.913242     0.0  0.882353  0.593315  0.935268  0.000000   
802    0.0  0.118721     0.0  0.870588  0.145775  0.104911  0.044776   
803    0.0  0.262557     0.0  0.541176  0.766017  0.241071  0.089552   
804    0.0  0.559361     0.0  0.717647  0.831012  0.569196  0.044776   
805    0.0  0.251142     0.0  0.682353  0.775302  0.267857  0.000000   

        일출 습도     일출 기압  일출 전운량       일교차  
0    0.913580  0.723481     0.0  0.432836  
1    0.901235  0.808704     0.7  0.656716  
2  

C:\Users\leeji\AppData\Local\Temp\ipykernel_14704\3720505129.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = scaler.fit_transform(df[[i]])
C:\Users\leeji\AppData\Local\Temp\ipykernel_14704\3720505129.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = scaler.fit_transform(df[[i]])
C:\Users\leeji\AppData\Local\Temp\ipykernel_14704\3720505129.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_model =DecisionTreeClassifier(max_depth=1)
clf =AdaBoostClassifier(base_estimator = tree_model, n_estimators =1000, random_state=0)
clf.fit(train_x, train_y)
pred=clf.predict(test_x)
print(accuracy_score(test_y, pred))

C:\Users\leeji\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.6956521739130435


In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from math import sqrt

clf2 = GradientBoostingClassifier(n_estimators=1000, random_state=0)
clf2.fit(train_x, train_y)
pred=clf2.predict(test_x)
print(accuracy_score(test_y, pred))

0.6832298136645962
